# Open Championship golf courses

#### Python tools

In [36]:
%load_ext lab_black

In [134]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_grid as altgrid
import fiona

In [135]:
alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")
fiona.supported_drivers["KML"] = "rw"

In [136]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

---

In [10]:
df = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vT52vRZvXL0PB33CiGEmg7Y-kwn2XnYM48k2LQ6VVRkPBr1uQbjeTSQwODvURuRSBCMJrgML9YpjuAv/pub?gid=0&single=true&output=csv"
)

In [11]:
df[["longitude", "latitude"]] = df["location"].str.split(",", expand=True)

In [12]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["latitude"], df["longitude"]))

In [13]:
gdf.to_file("data/raw/open_championship_courses.geojson", driver="GeoJSON")

/Users/stiles/.local/share/virtualenvs/grid-notebooks-Amv9DX-2/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


---

In [53]:
ids = pd.DataFrame(
    [
        {"erodeid": 11963, "course": "Royal Aberdeen", "founded": "1780"},
        {"erodeid": 4858, "course": "St. Andrews", "founded": "1552"},
        {"erodeid": 14109, "course": "Castle Stewart", "founded": "2009"},
        {"erodeid": 3434, "course": "North Berwick", "founded": "1832"},
        {"erodeid": 1325, "course": "Machrihanish", "founded": "1876"},
        {"erodeid": 9106, "course": "Montrose", "founded": "1562"},
    ]
)

In [54]:
ids

,erodeid,course,founded
0,11963,Royal Aberdeen,1780
1,4858,St. Andrews,1552
2,14109,Castle Stewart,2009
3,3434,North Berwick,1832
4,1325,Machrihanish,1876
5,9106,Montrose,1562


In [105]:
gdf = gpd.read_file(
    "/Users/stiles/data/climate/golf/layers/cagmap.snh.gov.uk/snh_dynamic_coast_two/Future Erosion 2100 High Emissions Scenario.geojson"
)

In [111]:
gdf.head()

,OBJECTID,LAYER,ERODETYPE,CELL,SUBCELL,CELL_SUB,ERODEID,GREEN_AREA,BTL_AREA,GDL_AREA,PIC_AREA,SHDMN_AREA,GOLF_AREA,ROAD_LEN,RAIL_LEN,COMSV_NUM,PVA_AREA,RNWY_AREA,UTIL_NUM,CWTR_LEN,SWTR_NUM,MPA_AREA,NNR_AREA,SAC_AREA,SPA_AREA,SSSI_AREA,SWGP_LEN,SWOP_NUM,SWSPS_AREA,SWSTW_AREA,SWO_NUM,SWRM_LEN,SHAPE_LENG,Shape_Length,Shape_Area,geometry
0,1,Scotland_ErodedArea_2100,ErodedArea,11,b,11b,23092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,34.668,34.668131,0,"POLYGON ((-1.33687 59.89892, -1.33688 59.89892..."
1,2,Scotland_ErodedArea_2100,ErodedArea,11,b,11b,23093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,1.585,1.584528,0,"POLYGON ((-1.32963 59.89889, -1.32962 59.89888..."
2,3,Scotland_ErodedArea_2100,ErodedArea,11,b,11b,23094,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.996,0.995713,0,"POLYGON ((-1.32963 59.89889, -1.32964 59.89889..."
3,4,Scotland_ErodedArea_2100,ErodedArea,11,b,11b,23095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,1.016,1.016444,0,"POLYGON ((-1.32964 59.89889, -1.32964 59.89889..."
4,5,Scotland_ErodedArea_2100,ErodedArea,11,b,11b,23096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,1.025,1.025045,0,"POLYGON ((-1.32964 59.89889, -1.32965 59.89889..."


In [112]:
erodeid = ids.erodeid.to_list()

In [113]:
gdf["GOLF_AREA_ACRES"] = gdf["GOLF_AREA"] / 4047

In [114]:
gdf["GOLF_AREA_ACRES"].sum()

1991.9833876945884

In [126]:
top_erosion_golf_area = (
    gdf[gdf["ERODETYPE"] == "ErodedArea"]
    .sort_values("GOLF_AREA_ACRES", ascending=False)
    .head(20)
)

In [127]:
our_courses = top_erosion_golf_area[top_erosion_golf_area["ERODEID"].isin(erodeid)][
    ["OBJECTID", "ERODEID", "ERODETYPE", "GOLF_AREA_ACRES", "geometry"]
]

In [128]:
our_courses

,OBJECTID,ERODEID,ERODETYPE,GOLF_AREA_ACRES,geometry
18753,18754,4858,ErodedArea,75.217724,"POLYGON ((-2.82179 56.36659, -2.82179 56.36659..."
5143,5144,11963,ErodedArea,56.792940,"POLYGON ((-2.06740 57.19894, -2.06742 57.19894..."
18782,18783,9106,ErodedArea,43.922876,"POLYGON ((-2.42944 56.74847, -2.42944 56.74846..."
21731,21732,14109,ErodedArea,39.097860,"POLYGON ((-4.11688 57.51925, -4.11687 57.51926..."


In [118]:
our_courses = gdf[gdf["ERODEID"].isin(erodeid)][
    ["OBJECTID", "ERODEID", "ERODETYPE", "GOLF_AREA_ACRES", "geometry"]
]

In [98]:
courses = gpd.GeoDataFrame(
    pd.merge(our_courses, ids, left_on="ERODEID", right_on="erodeid")
)

In [99]:
courses

,OBJECTID,ERODEID,ERODETYPE,GOLF_AREA_ACRES,geometry,erodeid,course,founded
0,5144,11963,ErodedArea,56.792940,"POLYGON ((-2.06740 57.19894, -2.06742 57.19894...",11963,Royal Aberdeen,1780
1,6938,1325,ErodedArea,0.000000,"POLYGON ((-5.72447 55.42533, -5.72446 55.42534...",1325,Machrihanish,1876
2,19141,1325,Vicinity,0.000000,"POLYGON ((-2.39757 55.96184, -2.39757 55.96185...",1325,Machrihanish,1876
3,19397,1325,Influence,0.000000,"POLYGON ((-2.87426 56.01279, -2.87425 56.01279...",1325,Machrihanish,1876
4,17355,3434,Vicinity,0.000000,"POLYGON ((-6.04382 56.69045, -6.04382 56.69046...",3434,North Berwick,1832
5,19466,3434,ErodedArea,0.000000,"POLYGON ((-2.75236 56.05964, -2.75235 56.05964...",3434,North Berwick,1832
6,27816,3434,Influence,0.000000,"POLYGON ((-5.67025 57.02730, -5.67024 57.02730...",3434,North Berwick,1832
7,18754,4858,ErodedArea,75.217724,"POLYGON ((-2.82179 56.36659, -2.82179 56.36659...",4858,St. Andrews,1552
8,19808,4858,Influence,0.000000,"POLYGON ((-3.24400 57.70605, -3.24401 57.70606...",4858,St. Andrews,1552
9,29985,4858,Vicinity,0.000000,"POLYGON ((-5.61359 57.40199, -5.61359 57.40199...",4858,St. Andrews,1552


In [130]:
courses.to_file("data/raw/golf/scottish_courses.geojson", driver="GeoJSON")
top_erosion_golf_area.to_file(
    "data/raw/golf/top_erosion_golf_areas.geojson", driver="GeoJSON"
)

In [137]:
courses.to_file("data/raw/golf/scottish_courses.kml", driver="KML")
top_erosion_golf_area.to_file("data/raw/golf/top_erosion_golf_areas.kml", driver="KML")

/Users/stiles/.local/share/virtualenvs/grid-notebooks-Amv9DX-2/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/Users/stiles/.local/share/virtualenvs/grid-notebooks-Amv9DX-2/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
